# Programación avanzada- UNICABA

## Ejercicio de cierre de unidad 2

### Analisis de trends musicales en streaming segun Billboard

Tipo: Texto (string)

In [1]:
######################-DICCIONARIO DE DATOS DEL DATASET-########### 

#1-Date: Tipo: Fecha (formato yyyy/mm/dd) #    Descripción: Fecha en la que se registró el ranking de la canción.
#2-Song: Tipo: Texto (string) #    Descripción: Nombre de la canción en el ranking.
#3-Artist: Tipo: Texto (string) #    Descripción: Nombre del artista o grupo musical (puede incluir “Featuring” si hay colaboraciones).
#4-Rank: Tipo: Numérico entero #    Descripción: Posición actual de la canción en el ranking (1 = primer lugar).
#5-Last_Week:Tipo: Numérico entero #    Descripción: Posición que ocupaba la canción en la semana anterior.
#6-Peak_Position:Tipo: Numérico entero #    Descripción: Mejor posición alcanzada por la canción en el ranking hasta la fecha.
#7-Weeks_in_Charts:Tipo: Numérico entero #    Descripción: Número de semanas que la canción lleva en el ranking.

In [2]:

import os
from supabase import create_client, Client
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Configuración 
SUPABASE_URL = os.environ["SUPABASE_URL"]
SUPABASE_KEY = os.environ["SUPABASE_KEY"]
TABLE_NAME = "Dataset_Ranking"
PAGE_SIZE = 1000  # Límite por defecto de Supabase
# ---------------------------------------------

# Inicialización
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
all_records = []
offset = 0

print(f"Iniciando descarga paginada de la tabla '{TABLE_NAME}'...")

while True:
    #  Definir el rango de filas a solicitar (paginación)
    start_row = offset
    end_row = offset + PAGE_SIZE - 1
    
    #print(f" -> Fetching filas {start_row} a {end_row}...")

    #  Ejecutar la consulta con el rango
    response = (
        supabase.table(TABLE_NAME)
        .select('*')
        .order('Date', desc=True) 
        .range(start_row, end_row) #  La clave de la paginación
        .execute()
    )
    
    data = response.data

    #  Romper el bucle si no hay más datos
    if not data:
        print("Descarga completa: No se encontraron más registros.")
        break
    
    #  Acumular los registros
    all_records.extend(data)
    
    #  Aumentar el desplazamiento para la siguiente página
    offset += PAGE_SIZE
    
#  Crear el DataFrame ÚNICO
if all_records:
    df = pd.DataFrame(all_records)
    print(f"\n  {len(df)} registros totales cargados en un solo DataFrame.")

else:
    print("La tabla está vacía.")

Iniciando descarga paginada de la tabla 'Dataset_Ranking'...
✅ Descarga completa: No se encontraron más registros.

  33050 registros totales cargados en un solo DataFrame.


In [4]:
# dataframe
#df = pd.read_csv("/work/streaming_songs.csv")
df.head(10)

,ID,Date,Song,Artist,Rank,Last_Week,Peak_Position,Weeks_in_Charts
0,8658,2025-09-24,Sports Car,Tate McRae,38,8,15,0
1,4237,2025-09-24,Folded,Kehlani,27,33,25,7
2,33046,2025-09-24,I Had Some Help,Post Malone Featuring Morgan Wallen,28,35,1,70
3,11461,2025-09-24,No One Noticed,The Marias,22,27,11,39
4,33045,2025-09-24,A Bar Song (Tipsy),Shaboozey,24,30,1,74
5,16034,2025-09-24,What It Sounds Like,HUNTR/X: EJAE| Audrey Nuna & REI AMI,6,7,6,12
6,5655,2025-09-24,Sugar On My Tongue,Tyler| the Creator,43,42,21,8
7,33040,2025-09-24,What I Want,Morgan Wallen Featuring Tate McRae,8,9,1,18
8,17424,2025-09-24,Creep,Radiohead,48,48,5,0
9,12229,2025-09-24,When Did You Get Hot?,Sabrina Carpenter,17,14,10,3


In [5]:
#encontrar vacios
df.isnull().sum()
df.info()
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33050 entries, 0 to 33049
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               33050 non-null  int64 
 1   Date             33050 non-null  object
 2   Song             33050 non-null  object
 3   Artist           33050 non-null  object
 4   Rank             33050 non-null  int64 
 5   Last_Week        33050 non-null  int64 
 6   Peak_Position    33050 non-null  int64 
 7   Weeks_in_Charts  33050 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.0+ MB
Filas: 33050, Columnas: 8


In [6]:
df.describe()


,ID,Rank,Last_Week,Peak_Position,Weeks_in_Charts
count,33050.000000,33050.000000,33050.000000,33050.000000,33050.000000
mean,16525.500000,25.500000,23.190439,10.174554,14.520938
std,9540.857535,14.431088,14.032791,11.959740,16.682715
min,1.000000,1.000000,1.000000,1.000000,0.000000
25%,8263.250000,13.000000,11.000000,1.000000,3.000000
50%,16525.500000,25.500000,22.000000,5.000000,9.000000
75%,24787.750000,38.000000,35.000000,15.000000,20.000000
max,33050.000000,50.000000,50.000000,148.000000,149.000000


Los datos fueron recolectados desde el 23/01/2013 hasta el 24/09/2025, contiene 33050 registros de canciones de streaming calculadas por Billboard

In [7]:
# columna Date a datetime
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")

# Extraer el año
df["Year"] = df["Date"].dt.year

# canciones por año

songs_per_year = df.groupby("Year")["Song"].nunique().reset_index()
songs_per_year.columns = ["Year", "UniqueSongCount"]

average_unique_songs_per_year = songs_per_year["UniqueSongCount"].mean()
print(f"Promedio de canciones únicas por año: {average_unique_songs_per_year:.2f}")

Promedio de canciones únicas por año: 363.46


In [8]:

# Graficar
fig = px.line(
    songs_per_year,
    x="Year",
    y="UniqueSongCount",
    markers=True,
    title="Evolución de canciones por año",
    labels={"Year": "Año", "UniqueSongCount": "Cantidad de canciones"}
)

fig.update_layout(template="plotly_white")
fig.show()

Hay una media de 343 canciones por año, se evidencia una mayor cantidad de canciones en streaming a medida que pasa el tiempo. Nota: la cantidad en el 2025 se debe a que solo se incluye hasta septiembre

In [9]:
# canciones únicas
unique_songs = df["Song"].nunique()

#  card 
fig = go.Figure(go.Indicator(
    mode="number",
    value=unique_songs,
    title={"text": " Canciones únicas en streaming"},
    number={"valueformat": ","}
))

fig.update_layout(
    height=200,
    margin=dict(t=30, b=0, l=0, r=0),
    template="plotly_white"
)

fig.show()



In [10]:

#  artistas únicos
unique_artists = df["Artist"].nunique()

#  card 
fig = go.Figure(go.Indicator(
    mode="number",
    value=unique_artists,
    title={"text": " Artistas únicos"},
    number={"valueformat": ","}
))

fig.update_layout(
    height=200,
    margin=dict(t=30, b=0, l=0, r=0),
    template="plotly_white"
)

fig.show()



In [11]:
# Filtrar solo las filas donde Peak Position es 1
top_hits = df[df["Peak_Position"] == 1]

#  #1 de cada artista
artist_counts = top_hits["Artist"].value_counts()

# artista con más #1
top_artist = artist_counts.idxmax()
top_count = artist_counts.max()
print(f"El artista con más semanas en el puesto #1 es: {top_artist} ({top_count})")
print(artist_counts.head(10))


El artista con más semanas en el puesto #1 es: Taylor Swift (348)
Artist
Taylor Swift              348
Drake                     303
Morgan Wallen             231
The Weeknd                170
Kendrick Lamar            166
Ariana Grande             155
Miley Cyrus               148
PSY                       123
Olivia Rodrigo            119
Post Malone & Swae Lee    117
Name: count, dtype: int64


In [12]:
# top 10 artistas
top_10_artists = artist_counts.head(10).index.tolist()

# Filtrar los datos de esos artistas
top_10_data = top_hits[top_hits["Artist"].isin(top_10_artists)]

# Agrupar por año y artista
artist_year_counts = top_10_data.groupby(["Year", "Artist"]).size().reset_index(name="NumberOneHits")
fig = px.bar(
    artist_year_counts,
    x="Year",
    y="NumberOneHits",
    color="Artist",
    barmode="group",
    title=" Evolución anual de los top 10 artistas con canciones en el puesto #1",
    labels={"Year": "Año", "NumberOneHits": "#1 por año"}
)

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    template="plotly_white",
    height=600
)

fig.show()


In [13]:
#  top 10 artistas
top_10_artists = artist_counts.head(5).index.tolist()

# Filtrar solo los datos de esos artistas
top_10_data = top_hits[top_hits["Artist"].isin(top_10_artists)]

# Agrupar por año y artista
artist_year_counts = top_10_data.groupby(["Year", "Artist"]).size().reset_index(name="NumberOneHits")
import plotly.express as px

fig = px.line(
    artist_year_counts,
    x="Year",
    y="NumberOneHits",
    color="Artist",
    markers=True,
    title=" Evolución temporal de los top 10 artistas con canciones en el puesto #1",
    labels={"Year": "Año", "NumberOneHits": "#1 por año"}
)

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    template="plotly_white",
    height=600
)

fig.show()


In [14]:
top_1 = df[df['Peak_Position'] == 1].copy()
# Extraer el año
top_1['year'] = top_1['Date'].dt.year

# Contar artistas únicos por año
artistas_por_anio = top_1.groupby('Year')['Artist'].nunique().reset_index()
artistas_por_anio.columns = ['año', 'artistas_unicos_en_posicion_1']

print(artistas_por_anio)

fig = px.line(
    artistas_por_anio,
    x='año',
    y='artistas_unicos_en_posicion_1',
    markers=True,
    title='Artistas únicos en la posición #1 por año',
    labels={
        'anio': 'Año',
        'artistas_unicos_en_posicion_1': 'Artistas únicos en #1'
    }
)

fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis=dict(title='Cantidad de artistas'),
    hovermode='x unified'
)

fig.show()



     año  artistas_unicos_en_posicion_1
0   2013                            182
1   2014                            140
2   2015                            124
3   2016                            144
4   2017                            166
5   2018                            200
6   2019                            198
7   2020                            253
8   2021                            239
9   2022                            229
10  2023                            197
11  2024                            199
12  2025                            132


In [15]:


# mejor posición que alcanzó cada canción
best_positions = df.groupby("Song", as_index=False)["Peak_Position"].min()

# Unir con los artistas 
song_artist = df.groupby("Song", as_index=False)["Artist"].first()

# Merge para tener artista + mejor posición
merged = pd.merge(best_positions, song_artist, on="Song")

# Filtrar canciones que llegaron al top 10
top_10_songs = merged[merged["Peak_Position"] <= 10]

#  artistas únicos que llegaron al top 10
top_10_artists = top_10_songs["Artist"].unique()

# Contar cuántos artistas son
num_top_10_artists = len(top_10_artists)

print(f" Total de artistas que llegaron alguna vez al top 10: {num_top_10_artists}")

 Total de artistas que llegaron alguna vez al top 10: 1700


In [16]:
# Revisar el error y corregir el código
# Parece que hay un error en la creación del DataFrame, ya que se está utilizando el índice en lugar de los valores de la columna "Artist".

# Corregir el código
# Crear un DataFrame con los 10 artistas con más semanas en la última posición
last_position_hits = df[df["Rank"] == 50]

# Contar el número de semanas que cada artista ha estado en la última posición
last_position_counts = last_position_hits["Artist"].value_counts().head(10)

# Crear un DataFrame para los 10 artistas con más semanas en la última posición
top_10_last_position = last_position_counts.reset_index()
top_10_last_position.columns = ["Artist", "Weeks in Last Position"]

# Graficar
fig = px.bar(
    top_10_last_position,
    x="Weeks in Last Position",
    y="Artist",
    orientation="h",
    title="Top 10 Artistas con más semanas en la última posición",
    text="Weeks in Last Position"
)

fig.update_traces(texttemplate='%{text}', textposition='inside')
fig.update_layout(
    xaxis_title="Semanas en la última posición",
    yaxis_title="Artista",
    template="plotly_white",
    height=600
)

fig.show()

Here is a bar chart showing the Top 10 artists who have spent the most weeks at the last position in the ranking.

Here is a bar chart showing the annual evolution of the top 4 artists with songs that reached the #1 position.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a2cd30de-aa34-492f-bd7f-baf47d78ec21' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>